In [16]:
from __future__ import print_function
%matplotlib ipympl
from typing import Tuple
import numpy as np

import math
from math import sin, cos

from os import listdir
from os.path import isfile, join


from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt


#----------- Constants ----------------
a = 10
# MAGIC_VALUE = 0.9945 
VALUE = 1
rot = 0.04884


#----------- Odometry function ----------------

def get_diff_in_cm(ROT_TO_CM, magic_value, tu1:Tuple[int,int], tu2:Tuple[int,int]) -> Tuple[float, float]:
    return((tu1[0]-tu2[0]) * ROT_TO_CM * magic_value, (tu1[1]-tu2[1]) * ROT_TO_CM / magic_value)



# this is the odomety function of the rover, using nupy array for plotting

def update_position(drive_data, AXLE_LENGTH, ROT_TO_CM, MAGIC_VALUE,
                    local_x_coordinat = 0, local_y_coordinat = 0, local_oriantation:float = 0):
    list_of_coords = []
    list_np = np.zeros((len(drive_data), 2))
    alpha = 0
    for i in range(1, len(drive_data)-1):
        dl, dr = get_diff_in_cm(ROT_TO_CM,MAGIC_VALUE, drive_data[i+1], drive_data[i])

        if dl == dr:
            alpha = 0
            s = dl
        else:
            alpha = (dr - dl) / AXLE_LENGTH
            s = AXLE_LENGTH * (dr + dl) / (dr - dl) * math.sin((dr - dl) / (2 * AXLE_LENGTH))


        local_oriantation = local_oriantation + alpha
            # print(alpha)

        delta_x = s * (-1) * sin(local_oriantation)
        delta_y = s * cos(local_oriantation)


        local_x_coordinat += delta_x
        local_y_coordinat += delta_y

        list_of_coords.append((local_x_coordinat, local_y_coordinat))
        list_np[i] = [local_x_coordinat, local_y_coordinat]
    return np.array(list_of_coords)

#plotting all file in folder test1 at the same time
#data needs to end i x.csv where x is the starting oriantation

test_data_folder = 'data/test1/'
data_files = [f for f in listdir(test_data_folder) if isfile(join(test_data_folder, f))]
dir_list = []
drive_data = []

#reading data from csv file
for file in data_files:
    dir_list.append(file[-5])
    drive_data.append(np.genfromtxt('data/test1/' + file, delimiter=','))

directions = {"N":0, "E":-math.pi/2, "S":math.pi, "W":math.pi/2}
    
def plot_data_interactive(A, R, M):
    plt.cla()
    for i in range(len(drive_data)):
        data = update_position(drive_data[i],A, R, M, local_oriantation=directions[dir_list[i]])


    #data = update_position(a, rot)
        plt.plot(data[:,0], data[:,1])

    plt.grid()
    plt.axis("scaled")
    plt.xlim((-300,300))
    plt.ylim((-300,300))
    plt.show()
    return()

interact(plot_data_interactive, 
         A=widgets.FloatSlider(min=5, max=20, step=0.1, value=10, readout_format='.1f'), 
         R=widgets.FloatSlider(min=0.04, max=0.06, step=0.001, value=0.047, readout_format='.3f'), 
         M=widgets.FloatSlider(min=0.99, max=1.01, step=0.001, value=0.998, readout_format='.3f'));


interactive(children=(FloatSlider(value=10.0, description='A', max=20.0, min=5.0, readout_format='.1f'), Float…